In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/capstone/MDMCGAN.zip -d /content/project

Archive:  /content/drive/MyDrive/capstone/MDMCGAN.zip
   creating: /content/project/MDMCGAN/
  inflating: /content/project/MDMCGAN/cond_wgan_gp.py  
  inflating: /content/project/MDMCGAN/.DS_Store  
  inflating: /content/project/__MACOSX/MDMCGAN/._.DS_Store  
  inflating: /content/project/MDMCGAN/requirements.txt  
  inflating: /content/project/MDMCGAN/train_params.py  
  inflating: /content/project/MDMCGAN/gra.py  
   creating: /content/project/MDMCGAN/preprocess/
   creating: /content/project/MDMCGAN/models/
   creating: /content/project/MDMCGAN/__pycache__/
   creating: /content/project/MDMCGAN/UCI_HAR/
  inflating: /content/project/MDMCGAN/mdmcgan.py  
  inflating: /content/project/MDMCGAN/README.md  
   creating: /content/project/MDMCGAN/generator/
  inflating: /content/project/MDMCGAN/.gitignore  
  inflating: /content/project/MDMCGAN/utils.py  
   creating: /content/project/MDMCGAN/.git/
  inflating: /content/project/MDMCGAN/score_3_0.9_result.csv  
  inflating: /content/project

In [4]:
!pip install -r /content/project/MDMCGAN/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!pip install tensorflow
!pip install matplotlib

In [3]:
import os
os.chdir('/content/project/MDMCGAN')

In [4]:
# Ignore unrecognized command-line arguments
import sys
if 'google.colab' in sys.modules:
    sys.argv = sys.argv[:1]

In [35]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import csv
import tensorflow as tf
import sys

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, roc_curve, RocCurveDisplay
from torch.utils.data import DataLoader, TensorDataset

# Assuming the following import paths are correct; you might need to adjust them
from models.mdmcgan import Generator as mdmcgan_gen
from models.cond_wgan_gp import Generator as cond_wgan_gp_gen
from train_params import opt


**Setting 여기서 만져주세요**

In [68]:
#Setting
datatype = "iid"
remove_labels_num = 3 #choose from 1,2,3,5
filter_ratio = 0.9
plot_name = "plots"

In [69]:
os.makedirs(plot_name, exist_ok=True)

mixed_precision.set_global_policy('mixed_float16')

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [70]:
if torch.cuda.is_available():
    print("GPU is available")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available")

GPU is available
GPU Name: Tesla T4


In [71]:
iid_labels = 1500
non_iid_labels = 1500
ratio = 18

In [72]:
def flatten_features(np_arr):
    np_arr = np.squeeze(np_arr)
    np_arr = np.concatenate(np.moveaxis(np_arr, 2, 0), axis=1)
    return np_arr

def get_generator(algorithm, non_iid):
    if algorithm == "orig_cond_wgan_gp":
        cur_generators = []
        for i in range(opt.num_modalities):
            cur_generators.append(cond_wgan_gp_gen(opt).to(device))
            if non_iid == "non_iid":
                cur_generators[i].load_state_dict(
                    torch.load(
                        f"generator/{non_iid}_{remove_labels_num}_{filter_ratio}_{algorithm}_{i}",
                        map_location=torch.device("cpu"),
                    )
                )
            else:
                cur_generators[i].load_state_dict(
                    torch.load(
                        f"generator/{non_iid}_{algorithm}_{i}",
                        map_location=torch.device("cpu"),
                    )
                )
        return cur_generators
    else:
        cur_generator = (
            mdmcgan_gen(opt).to(device) if algorithm == "mdmcgan" else cond_wgan_gp_gen(opt).to(device)
        )
        if non_iid == "non_iid":
            cur_generator.load_state_dict(
                torch.load(
                    f"generator/{non_iid}_{remove_labels_num}_{filter_ratio}_{algorithm}",
                    map_location=torch.device("cpu"),
                )
            )
        else:
                cur_generator.load_state_dict(
                torch.load(
                    f"generator/{non_iid}_{algorithm}",
                    map_location=torch.device("cpu"),
                )
            )
        return cur_generator

def gen_fake_features(algorithm, labels):
    z = Tensor(np.random.normal(0, 1, (len(labels), opt.latent_dim))).to(device)
    labels = labels.to(device)

    with torch.no_grad():
        if algorithm == "mdmcgan":
            modals = []
            for i in range(opt.num_modalities):
                modals.append(torch.full((len(labels),), i, device=device))

            gen_features_list = []
            for cur_modal in modals:
                gen_features_list.append(generator(z, labels, cur_modal))

            gen_features = torch.cat(gen_features_list, dim=3)

        elif algorithm == "cond_wgan_gp":
            gen_features = generator(z, labels)
            gen_copy = gen_features.clone().detach()
            gen_features = torch.cat([gen_features, gen_copy], dim=3)
        else:
            gen_features_list = []
            for i in range(opt.num_modalities):
                gen_features_list.append(generators[i](z, labels))
            gen_features = torch.cat(gen_features_list, dim=3)
    return gen_features.cpu()

In [73]:
def gen_blended_features(algorithm, non_iid_str):
    uci_har_folder = "UCI_HAR/filtered_data"

    if non_iid_str == "non_iid":
        data = np.load(
            f"{uci_har_folder}/{non_iid_str}_{remove_labels_num}_{filter_ratio}.npz"
        )
    else:
        data = np.load(f"{uci_har_folder}/{non_iid_str}.npz")

    X_train = flatten_features(data["X_train"])
    X_test = flatten_features(data["X_test"])
    Y_train = data["Y_train"]
    Y_test = data["Y_test"]

    if non_iid_str == "non_iid":
        y_remain_gyro = data["y_remain_gyro"]
        gyro_remain = data["gyro_remain"]
        y_remain_acc = data["y_remain_acc"]
        acc_remain = data["acc_remain"]

    data.close()

    if non_iid_str == "non_iid":
        gen_acc = gen_fake_features(algorithm, torch.from_numpy(y_remain_gyro))
        gen_acc = np.split(gen_acc.numpy(), 2, axis=3)[0]

        gen_gyro = gen_fake_features(algorithm, torch.from_numpy(y_remain_acc))
        gen_gyro = np.split(gen_gyro.numpy(), 2, axis=3)[1]

        gen_y = np.concatenate([y_remain_gyro, y_remain_acc], axis=0)

        gen_x = np.concatenate(
            [
                np.concatenate([gen_acc, gyro_remain], axis=3),
                np.concatenate([acc_remain, gen_gyro], axis=3),
            ],
            axis=0,
        )
        gen_x = flatten_features(gen_x)

    gen_labels = [
        i
        for i in range(6)
        for _ in range(non_iid_labels if non_iid_str == "non_iid" else iid_labels)
    ]

    gen_data = gen_fake_features(algorithm, torch.tensor(gen_labels))
    gen_data = flatten_features(gen_data.numpy())

    if non_iid_str == "iid":
        gen_x = gen_data
        gen_y = gen_labels

    X_train = np.concatenate([X_train, gen_x, gen_data], axis=0)
    Y_train = np.concatenate([Y_train, gen_y, gen_labels], axis=0)

    shuffler = np.random.permutation(len(X_train))
    X_train = X_train[shuffler]
    Y_train = Y_train[shuffler]

    return X_train, Y_train, X_test, Y_test

In [74]:
# Function to create LSTM model
def create_lstm_model(input_shape, params):
    model = Sequential([
        LSTM(params['lstm_units_1'], input_shape=input_shape, return_sequences=True, dropout=params['dropout_rate'], recurrent_dropout=params['recurrent_dropout']),
        Dropout(params['dropout_rate']),
        LSTM(params['lstm_units_2'], dropout=params['dropout_rate'], recurrent_dropout=params['recurrent_dropout']),
        BatchNormalization() if params['use_batch_norm'] else None,
        Dropout(params['dropout_rate']),
        Dense(6, activation=params['activation_function'])  # Assuming 6 classes in Y_train
    ])
    optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [87]:
# Hyperparameters
base_lr = 0.001
base_batch_size = 32
batch_size = 128
epoch = 10
params = {
    "epochs": epoch,
    "batch_size": batch_size,
    "learning_rate": base_lr * (batch_size / base_batch_size),
    "lstm_units_1": 64,
    "lstm_units_2": 64,
    "dropout_rate": 0.5,
    "use_batch_norm": True,
    "activation_function": 'softmax',
    "recurrent_dropout": 0,
    "optimizer_choice": 'adam',  # Can implement logic to switch optimizers if needed
    "learning_rate_decay": 0.99
}

In [88]:
algorithms = ["mdmcgan", "orig_cond_wgan_gp", "cond_wgan_gp"]
scoring = ["f1_weighted", "accuracy", "f1_macro"]

# Placeholder for generators list
generators = [None, None, None]

In [89]:
if datatype == "iid":
    result_f = open(f"score_iid_result.csv", "w")
else:
    result_f = open(f"score_{remove_labels_num}_{filter_ratio}_result.csv", "w")
writer = csv.writer(result_f)
writer.writerow(["algorithm", "f1_weighted", "f1_macro", "accuracy"])

41

In [90]:
for non_iid_str in [datatype]:
    for i, algorithm in enumerate(algorithms):
        generator = None

        if algorithm == "orig_cond_wgan_gp":
            generators = get_generator(algorithm, non_iid_str)
        else:
            generator = get_generator(algorithm, non_iid_str)

        print(f"{non_iid_str}_{algorithm}")

        X_train, Y_train, X_test, Y_test = gen_blended_features(algorithm, non_iid_str)

        # One-hot encode the labels for training the LSTM
        Y_train_onehot = np.eye(6)[Y_train].astype(int)
        Y_test_onehot = np.eye(6)[Y_test].astype(int)

        # Reshape input data to be 3D [samples, timesteps, features]
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        input_shape = (X_train.shape[1], X_train.shape[2])

        lstm_model = create_lstm_model(input_shape, params)

        # Train the LSTM model
        #early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        #lstm_model.fit(X_train, Y_train_onehot, epochs=params["epochs"], batch_size=params["batch_size"], validation_split=0.2, callbacks=[early_stopping])
        lstm_model.fit(X_train, Y_train_onehot, epochs=params["epochs"], batch_size=params["batch_size"], validation_split=0.2)

        predicted_prob = lstm_model.predict(X_test)
        predicted = np.argmax(predicted_prob, axis=1)

        f1_weighted = round(f1_score(Y_test, predicted, average="weighted"), 4)
        f1_macro = round(f1_score(Y_test, predicted, average="macro"), 4)
        accuracy = round(accuracy_score(Y_test, predicted), 4)

        print(f"{f1_weighted}, {f1_macro}, {accuracy}")

        writer.writerow(
            [f"{non_iid_str}_{algorithm}", f1_weighted, f1_macro, accuracy]
        )
        result_f.flush()

result_f.close()

iid_mdmcgan
Epoch 1/10
164/164 [==============================] - 15s 57ms/step - loss: 1.3305 - accuracy: 0.4362 - val_loss: 1.2211 - val_accuracy: 0.3390
Epoch 2/10
164/164 [==============================] - 9s 55ms/step - loss: 0.9280 - accuracy: 0.5166 - val_loss: 0.9062 - val_accuracy: 0.5903
Epoch 3/10
164/164 [==============================] - 9s 56ms/step - loss: 0.8807 - accuracy: 0.5279 - val_loss: 0.8108 - val_accuracy: 0.5665
Epoch 4/10
164/164 [==============================] - 8s 51ms/step - loss: 0.8702 - accuracy: 0.5433 - val_loss: 0.8237 - val_accuracy: 0.5707
Epoch 5/10
164/164 [==============================] - 9s 55ms/step - loss: 0.8389 - accuracy: 0.5564 - val_loss: 0.7954 - val_accuracy: 0.6181
Epoch 6/10
164/164 [==============================] - 8s 49ms/step - loss: 0.8054 - accuracy: 0.5772 - val_loss: 0.7603 - val_accuracy: 0.5880
Epoch 7/10
164/164 [==============================] - 9s 56ms/step - loss: 0.7777 - accuracy: 0.5905 - val_loss: 0.7997 - val_acc

In [ ]:
Y_test_onehot = np.eye(6)[Y_test].astype(int)
colors = ["blue", "darkorange", "teal"]

for j in range(6):
    fig = plt.figure(figsize=(8, 8))
    fig.set_facecolor("white")
    ax = fig.add_subplot()
    ax.set_title(f"label {j} vs rest")

    for i, algorithm in enumerate(algorithms):
        predicted_prob = lstm_model.predict(X_test)

        RocCurveDisplay.from_predictions(
            Y_test_onehot[:, j],
            predicted_prob[:, j],
            name=algorithm,
            color=colors[i],
            ax=ax,
        )

    ax.plot([0, 1], [0, 1], color="red", label="Chance Level")
    ax.legend()

    plt.savefig(f""+plot_name+"/label_{j}.png")